In [12]:

import os
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
import time
from dotenv import load_dotenv
import bs4
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader

load_dotenv()
from langchain_community.document_loaders import PyPDFDirectoryLoader
## load the Groq API key
groq_api_key = os.environ['GROQ_API_KEY']

In [42]:
# from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# model_name = "BAAI/bge-large-en"
# model_kwargs = {"device": "cpu"}
# encode_kwargs = {"normalize_embeddings": True}
# hf = HuggingFaceBgeEmbeddings(
#     model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
# )

c:\Users\Ritesh Kapoor\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader


In [6]:
embeddings=OllamaEmbeddings()
url = "https://www.datacamp.com/cheat-sheet/getting-started-with-python-cheat-sheet"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

print(docs)
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=250) 
final_documents=text_splitter.split_documents(docs) 
vectors=FAISS.from_documents(final_documents,embeddings) 


[Document(page_content='Just a moment...Enable JavaScript and cookies to continue', metadata={'source': 'https://www.datacamp.com/cheat-sheet/getting-started-with-python-cheat-sheet', 'content_type': 'text/html; charset=UTF-8', 'title': 'Just a moment...', 'language': 'en-US'})]


In [7]:

llm=ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")


prompt=ChatPromptTemplate.from_template(
"""
you are a coding test platform.
I have give you a context of python documentation. 
You need to ask questions only on python language  <context>
{context}
<context>
Questions:{input}

"""
)
document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vectors.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)



In [8]:

response=retrieval_chain.invoke({"input":'create 1 random subjective type question on python language generate its answer also     '})

print(response['answer'])

Question: 

Without actually running the code, can you determine what the output of the following Python code will be and explain the steps that the Python interpreter takes to execute it?
```python
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

print(factorial(5))
```
Answer:

The output of the above code will be `120`.

Explanation:

1. When the Python interpreter encounters the `print` statement, it will call the `factorial` function with the argument `5`.
2. The `factorial` function checks if `n` is equal to `0`. If it is, the function returns `1`.
3. In this case, `n` is not equal to `0`, so the function returns the product of `n` and the result of calling `factorial` recursively with the argument `n-1`.
4. This process repeats until `n` is equal to `0`, at which point the recursion stops and the function returns `1`.
5. The recursive calls to `factorial` unwind, and the intermediate results are multiplied together. Specifically, `5 

In [9]:
from llama_index.llms.groq import Groq
from dotenv import load_dotenv
import os 
load_dotenv()

## load the Groq API key
groq_api_key = os.environ['GROQ_API_KEY']
llm = Groq(model="mixtral-8x7b-32768", api_key=groq_api_key)

c:\Users\Ritesh Kapoor\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from llama_index.core.llms import ChatMessage
answer=response
messages = [
    ChatMessage(
        role="system", content="you are an intelligent ai assistant.check the answer given. If answer has some errors then make it better  "
    ),
    ChatMessage(role="user", content=f"{answer}"),
]
resp = llm.chat(messages)
print(resp)

assistant: The answer is mostly correct, but there is a small mistake in the explanation step 4. The function does not return the product of n and the result of calling factorial recursively with the argument n-1 right away. It first checks if n-1 is equal to 0, if it is, it returns 1, otherwise, it continues the recursion.

Here is the corrected answer:

Question:

Without actually running the code, can you determine what the output of the following Python code will be and explain the steps that the Python interpreter takes to execute it?
```python
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

print(factorial(5))
```
Answer:

The output of the above code will be `120`.

Explanation:

1. When the Python interpreter encounters the `print` statement, it will call the `factorial` function with the argument `5`.
2. The `factorial` function checks if `n` is equal to `0`. If it is, the function returns `1`.
3. In this case, `n` is not equal to